In [2]:
# One Pickle for SNP

# STATUS: WIP
# Run-time:

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 1300, clientId=2)

In [44]:
%%time
import pandas as pd
import numpy as np
import itertools
import datetime
from math import sqrt, exp, log, erf
import os

#... assignments
exchange = 'SMART'
currency = 'USD'
fspath = './zdata/'

putsigma = 1.5
callsigma = 2
maxdte = 70  # max expiry date for options
mindte = 20  # min expiry date for options

tradingdays = 252

blks = 50

#... Risk-free rate assumed
rate_url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield'
df_rate = pd.read_html(rate_url)[1]
df_rate.columns  = df_rate.iloc[0] # Set the first row as header
df_rate = df_rate.drop(0,0) # Drop the first row
rate = float(df_rate[-1:]['1 yr'].values[0])/100 # Get the last row's 1 yr value as float


#... Functions
#_____________

#... Error catching for list comprehension

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
        
#...function to get historical data
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    
    df = util.df(hist)
    df.insert(0, column='symbol', value=contract.symbol)
    
    return df

#...function to get price and dividend ticker
def get_dividend_ticker(contract):
    '''Gets dividend ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.dividends is None:
        ib.sleep(1)

    ib.cancelMktData(contract)
       
    return ticker

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatility, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatility)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatility
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatility**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format yyyymmdd
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

# ... build the snp list

sym_chg_dict = {'BRK.B': 'BRK B', 'BRK/B': 'BRK B'} # Remap symbols in line with IBKR

snpurl = 'https://en.wikipedia.org/wiki/S%26P_100'
df_snp = pd.read_html(snpurl, header=0)[2]

df_snp.Symbol = df_snp.Symbol.map(sym_chg_dict).fillna(df_snp.Symbol)
df_snp['Type'] = 'Stock'

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

# read from row no 11, dropna and reset index
df_cboe = pd.read_excel(dls, header=12, 
                        usecols=[0,2,3]).loc[11:, :]\
                        .reset_index(drop=True)

# remove column names white-spaces and remap to IBKR
df_cboe.columns = df_cboe.columns.str.replace(' ', '')
df_cboe.Ticker = df_cboe.Ticker.map(sym_chg_dict).fillna(df_cboe.Ticker)

# list the equities
equities = [e for e in list(df_snp.Symbol) if e in list(df_cboe.Ticker)]

# filter and list the etfs
df_etf = df_cboe[df_cboe.ProductType == 'ETF'].reset_index(drop=True)
etfs = list(df_etf.Ticker)

stocks = sorted(equities+etfs)

# list the indexes
indexes = sorted('OEX,XEO,XSP,DJX'.split(','))

# Build a list of contracts
ss = [Stock(symbol=s, currency=currency, exchange=exchange) for s in set(stocks)]
# ixs = [Index(symbol=s,currency=currency, exchange='CBOE') for s in set(indexes)]
ixs = [Index(symbol=s,currency=currency, exchange=exchange) for s in set(indexes)]

cs = ss+ixs

# sort in alphabetical order
cs.sort(key=lambda x: x.symbol, reverse=False)

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
contract = next(q for q in qcs if q.symbol=='AAPL')  # one symbol logic check DATA LIMITER!!!
#___________________________________________________________________________

#... get ohlc, with cumulative volatality and standard deviation
#_______________________________________________________________

df_ohlc = get_hist(contract, 365).set_index('date').sort_index(ascending = False)

# get cumulative standard deviation
df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
df_stdev.columns = ['stdev']

# get cumulative volatility
df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
df_vol.columns = ['volatility']

df_ohlc1 = df_ohlc.join(df_vol)

df_ohlc2 = df_ohlc1.join(df_stdev)

#pickle the ohlc
df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')

#... get the underlyings
#_______________________

ticker = get_dividend_ticker(contract)

df_und = util.df([ticker])

cols = ['contract', 'time', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 
        'volume', 'open', 'high', 'low', 'close', 'dividends']
df_und = df_und[cols]

df_und = df_und.assign(undPrice=np.where(df_und['last'].isnull(), df_und.close, df_und['last']))

try: 
    divrate = df_und.dividends[0][0]/df_und.dividends[0][0]/df_und.undPrice
except (TypeError, AttributeError) as e:
    divrate = 0.0

df_und = df_und.assign(divrate=divrate)

df_und = df_und.assign(symbol=[c[1].symbol for c in df_und.contract.items()])

#... get the lot, margin, undPrice and dividend rate
undlot = 100

# margin of underlying
order = Order(action='SELL', totalQuantity=undlot, orderType='MKT')

margin = float(ib.whatIfOrder(contract, order).initMarginChange)

df_und['margin'] = margin

df_und.to_pickle(fspath+contract.symbol+'_und.pkl')

#... get the options
#___________________

# symbol
symbol = contract.symbol

# rights
right = ['P', 'C'] 

# chains
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

chain = next(c for c in chains if c.exchange == exchange)

undPrice = df_und.undPrice[0]

strikes = sorted([strike for strike in chain.strikes])
# limit the strikes to outside sigma range
# strikes = sorted([strike for strike in chain.strikes 
#            if (strike < (undPrice - std*putsigma)) | (strike > (undPrice + std*callsigma))])

# limit the expirations to between min and max dates
expirations = sorted([exp for exp in chain.expirations 
                      if mindte < (util.parseIBDatetime(exp)- datetime.datetime.now().date()).days < maxdte])

rights = ['P', 'C']

contracts = [Option(symbol, expiration, strike, right, exchange)
        for right in rights
        for expiration in expirations
        for strike in strikes]

# Eliminate contracts close to the underlying price
dtes = [(util.parseIBDatetime(e) - datetime.datetime.now().date()).days for e in expirations]
sdevs = [df_ohlc2.iloc[i].stdev for i in dtes]


tgts = []
for c in contracts:
    if c.right == 'P':
        if c.strike < (undPrice - df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * putsigma):
            tgts.append(c)
    else:
        if c.strike > (undPrice + df_ohlc2.iloc[get_dte(c.lastTradeDateOrContractMonth)].stdev * callsigma):
            tgts.append(c)

[ib.qualifyContracts(*tgts[i: i+blks]) for i in range(0, len(contracts), blks)]

qc = [c for c in tgts if c.conId != 0]  # qualified option contracts

df_opt = pd.concat([pd.DataFrame(qc, columns=['option']), util.df(qc)], axis=1)

df_opt1 = df_opt[['symbol', 'strike', 'lastTradeDateOrContractMonth', 'right', 'multiplier', 'exchange', 'option']]  # remove unnecessary columns

df_opt1 = df_opt1.rename(columns={'lastTradeDateOrContractMonth': 'expiry'}) # rename expiry column

df_opt1 = df_opt1.assign(undPrice=df_opt1.symbol.map(df_und.set_index('symbol')['undPrice']))

df_opt1['dte'] = (df_opt1.expiry.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

df_opt1.loc[df_opt1.dte <= 1, 'dte'] = 2 # Make the dte as 2 for 1 day-to-expiry to prevent bsm divide-by-zero error

# get the standard deviation based on days to expiry
df_opt1 = df_opt1.assign(stdev=[df_ohlc2.iloc[i].stdev for i in df_opt1.dte])

# weed out options within threshold of strike and dte
mask = (((df_opt1.right == 'P') & (df_opt1.strike < (df_opt1.undPrice - df_opt1.stdev * putsigma))) | \
       ((df_opt1.right == 'C') & (df_opt1.strike > (df_opt1.undPrice + df_opt1.stdev * callsigma))))

df_opt1 = df_opt1.loc[mask, :].reset_index(drop=True)

# get the volatality based on days to expiry
df_opt1 = df_opt1.assign(volatility=[df_ohlc2.iloc[i].volatility for i in df_opt1.dte])

# get the divrate
divrate = df_und.divrate[0]

bsms = [get_bsm(undPrice, strike, dte, rate, volatility, divrate) 
        for undPrice, strike, dte, rate, volatility, divrate in 
        zip(itertools.repeat(undPrice), df_opt1.strike, df_opt1.dte, itertools.repeat(rate), df_opt1.volatility, itertools.repeat(divrate))]

df_bsm = pd.DataFrame(bsms)

df_opt2 = df_opt1.join(df_bsm)
df_opt2['bsmPrice'] = np.where(df_opt2.right == 'P', df_opt2.bsmPut, df_opt2.bsmCall)
df_opt2['pop'] = np.where(df_opt2.right == 'C', 1-df_opt2.bsmDelta, df_opt2.bsmDelta)
df_opt2 = df_opt2.drop(['bsmCall', 'bsmPut', 'bsmDelta'], axis=1)

# get the option prices
cs = list(df_opt2.option)

# [catch(lambda: ib.reqTickers(c).marketPrice()) for i in range(0, len(cs), 100) for c in cs[i: i+100]]
tickers = [ib.reqTickers(*cs[i: i+100]) for i in range(0, len(cs), 100)]

df_opt3 = df_opt2.assign(price=[t.marketPrice() for ts in tickers for t in ts])

# filter the options whose price is > 0.0
df_opt4 = df_opt3[df_opt3.price > 0.0]

# get the margin of options and make return-on-margin
co = Order(action='SELL', totalQuantity=1, orderType='MKT')
margins = [float(ib.whatIfOrder(contract, order).initMarginChange) for contract, order in zip(df_opt4.option, itertools.repeat(co))]

df_opt4 = df_opt4.assign(margin=margins)

df_opt4 = df_opt4.assign(rom=df_opt4.price/df_opt4.margin*tradingdays/df_opt4.dte*df_opt4.multiplier.astype('float'))

df_opt4 = df_opt4.sort_values(by='rom', ascending=False)

df_opt4.to_pickle(fspath+contract.symbol+'_opt.pkl')

Started to throttle requests
Error 200, reqId 18623: No security definition has been found for the request, contract: Index(symbol='DJX', exchange='SMART', currency='USD')
Stopped to throttle requests
Error 200, reqId 18701: No security definition has been found for the request, contract: Index(symbol='OEX', exchange='SMART', currency='USD')
Error 200, reqId 18753: No security definition has been found for the request, contract: Index(symbol='XEO', exchange='SMART', currency='USD')
Error 200, reqId 18746: No security definition has been found for the request, contract: Stock(symbol='VXX', exchange='SMART', currency='USD')
Error 200, reqId 18768: No security definition has been found for the request, contract: Index(symbol='XSP', exchange='SMART', currency='USD')
Unknown contract: Index(symbol='DJX', exchange='SMART', currency='USD')
Unknown contract: Index(symbol='OEX', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='VXX', exchange='SMART', currency='USD')
Unknown con

Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=17.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=20.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=22.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=25.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=30.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=35.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=40.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190301', strike=45.0, right='P', exchange='SMART')
Unknown contract

Stopped to throttle requests
Error 200, reqId 18862: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=127.0, right='P', exchange='SMART')
Error 200, reqId 18863: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=128.0, right='P', exchange='SMART')
Error 200, reqId 18864: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=129.0, right='P', exchange='SMART')
Error 200, reqId 18866: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=131.0, right='P', exchange='SMART')
Error 200, reqId 18867: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=132.0, right='P', exchange='SMAR

Error 200, reqId 18903: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=50.0, right='P', exchange='SMART')
Error 200, reqId 18919: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=126.0, right='P', exchange='SMART')
Error 200, reqId 18920: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=127.0, right='P', exchange='SMART')
Error 200, reqId 18921: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=128.0, right='P', exchange='SMART')
Error 200, reqId 18922: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=129.0, right='P', exchange='SMART')
Unknown contract: Option(s

Error 200, reqId 18953: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=22.5, right='P', exchange='SMART')
Error 200, reqId 18954: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=25.0, right='P', exchange='SMART')
Error 200, reqId 18955: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 18956: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 18957: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 18958: No sec

...

Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=325.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=330.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=335.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=340.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=345.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=350.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=355.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190308', strike=360.0, right='C', exchange='SMART')
Unknown 

Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=207.5, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=305.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=310.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=315.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=320.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=325.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=330.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190315', strike=335.0, right='C', exchange='SMART')
Unknown 

Error 200, reqId 19200: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=325.0, right='C', exchange='SMART')
Error 200, reqId 19201: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=330.0, right='C', exchange='SMART')
Error 200, reqId 19202: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=335.0, right='C', exchange='SMART')
Error 200, reqId 19203: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=340.0, right='C', exchange='SMART')
Error 200, reqId 19204: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=345.0, right='C', exchange='SMART')
Error 200, reqId 19205: N

Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=365.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=370.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=375.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=380.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=385.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=390.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=395.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20190322', strike=400.0, right='C', exchange='SMART')
Unknown 

Wall time: 1min 15s


In [45]:
ib.disconnect()

In [47]:
df_opt4[(df_opt4['pop'] > 0.9) & (df_opt4.rom > 0.8)]

,symbol,strike,expiry,right,multiplier,exchange,option,undPrice,dte,stdev,volatility,bsmPrice,pop,price,margin,rom
70,AAPL,120.0,20190322,P,100,SMART,"Option(conId=351949760, symbol='AAPL', lastTra...",166.44,46,10.229351,0.455541,0.17,0.982957,0.33,97.02,1.863354
73,AAPL,135.0,20190322,P,100,SMART,"Option(conId=351949820, symbol='AAPL', lastTra...",166.44,46,10.229351,0.455541,1.08,0.917845,0.57,176.99,1.764285
72,AAPL,130.0,20190322,P,100,SMART,"Option(conId=351949799, symbol='AAPL', lastTra...",166.44,46,10.229351,0.455541,0.63,0.947822,0.45,142.65,1.728158
71,AAPL,125.0,20190322,P,100,SMART,"Option(conId=351949777, symbol='AAPL', lastTra...",166.44,46,10.229351,0.455541,0.34,0.969023,0.33,117.23,1.542119
14,AAPL,143.0,20190301,P,100,SMART,"Option(conId=349257946, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.94,0.906064,0.32,221.80,1.454283
13,AAPL,142.0,20190301,P,100,SMART,"Option(conId=349257921, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.83,0.915350,0.30,209.67,1.442266
12,AAPL,141.0,20190301,P,100,SMART,"Option(conId=349257910, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.73,0.923988,0.27,199.99,1.360868
11,AAPL,140.0,20190301,P,100,SMART,"Option(conId=349257907, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.64,0.931993,0.24,190.70,1.268589
10,AAPL,139.0,20190301,P,100,SMART,"Option(conId=349257895, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.56,0.939380,0.22,181.42,1.222357
9,AAPL,138.0,20190301,P,100,SMART,"Option(conId=349257886, symbol='AAPL', lastTra...",166.44,25,5.309310,0.465880,0.48,0.946171,0.19,173.21,1.105710
